In [19]:
import tensorflow as tf

from tensorflow.keras import layers, models
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

import random

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [20]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [21]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [22]:
def build_model():
    return models.Sequential([
        layers.Flatten(input_shape=(28, 28)), 
        layers.Dense(128, activation='relu'), 
        layers.Dense(64, activation='relu'),   
        layers.Dense(10)                      
    ])

In [23]:
def plot_history(history):
    fig, ax = plt.subplots(1, 5, figsize=(15, 5))
    fig.tight_layout(pad=5.0)

    ax[0].plot(history.history['accuracy'], label='accuracy')
    ax[0].plot(history.history['val_accuracy'], label='val_accuracy')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Accuracy')
    ax[0].set_ylim([0, 1])

    ax[1].plot(history.history['precision'], label='precision')
    ax[1].plot(history.history['val_precision'], label='val_precision')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Precision')
    ax[1].set_ylim([0, 1])

    ax[2].plot(history.history['recall'], label='recall')
    ax[2].plot(history.history['val_recall'], label='recall')
    ax[2].set_xlabel('Epoch')
    ax[2].set_ylabel('Recall')
    ax[2].set_ylim([0, 1])

    history.history['f1_score'] = (2 * np.array(history.history['precision']) * np.array(history.history['recall']) /
                                    (np.array(history.history['precision']) + np.array(history.history['recall']))).tolist()

    history.history['val_f1_score'] = (2 * np.array(history.history['val_precision']) * np.array(history.history['val_recall']) /
                                    (np.array(history.history['val_precision']) + np.array(history.history['val_recall']))).tolist()

    ax[3].plot(history.history['f1_score'], label='f1_score')
    ax[3].plot(history.history['val_f1_score'], label='val_f1_score')
    ax[3].set_xlabel('Epoch')
    ax[3].set_ylabel('F1Score')
    ax[3].set_ylim([0, 1])

    ax[4].plot(history.history['loss'], label='accuracy')
    ax[4].plot(history.history['val_loss'], label='val_accuracy')
    ax[4].set_xlabel('Epoch')
    ax[4].set_ylabel('Loss')
    ax[4].set_ylim([0, 3])

    plt.show()

In [24]:
def custom_prec_score(y_true, y_pred):
    y_true=y_true.numpy()
    y_pred=y_pred.numpy()
    y_pred=np.argmax(y_pred, axis=-1)
    return precision_score(y_true, y_pred,average='macro')

def recall(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.cast(tf.math.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), tf.float32))
    actual_positives = tf.reduce_sum(tf.cast(tf.equal(y_true, 1), tf.float32))
    return true_positives / (actual_positives + tf.keras.backend.epsilon())

def f1_score(y_true, y_pred):
    precision_value = precision(y_true, y_pred)
    recall_value = recall(y_true, y_pred)
    return 2 * ((precision_value * recall_value) / (precision_value + recall_value + tf.keras.backend.epsilon()))

In [25]:
def test_model(model, optimizer):
    model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[
                  'accuracy', 
                  precision,
                  recall, 
                  f1_score,
                ])
    
    print(f'\nTesting: {optimizer}')
    history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), verbose=False)
    
    reached_90 = next((i + 1 for i, e in enumerate(history.history["val_accuracy"]) if e >= 0.9), np.inf) 

    if reached_90 == np.inf:
        print('\nNever reached 90% accuracy')
    else:
        print(f'\nReached 90% accuracy in {reached_90} epochs')

    
    results = model.evaluate(test_images, test_labels, verbose=2)
    print(f'\nTest accuracy: {results[1]}, Test loss: {results[0]}')

    plot_history(history)
    print('\n\n\n')

    return (history, model)


Testing: SGD

Never reached 90% accuracy
313/313 [==============================] - 0s 793us/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.11      1.00      0.20      1135
           2       0.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

    accuracy                           0.11     10000
   macro avg       0.01      0.10      0.02     10000
weighted avg       0.01      0.11      0.02     10000


/home/volodymyr/.local/share/virtualenvs/python_neural_nets-VlmVW8YE/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/volodymyr/.local/share/virtualenvs/python_neural_nets-VlmVW8YE/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/volodymyr/.local/share/virtualenvs/python_neural_nets-VlmVW8YE/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. U


Testing: Adam

Reached 90% accuracy in 1 epochs
313/313 [==============================] - 0s 818us/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.97      0.97      0.97      1032
           3       0.95      0.98      0.97      1010
           4       0.98      0.97      0.97       982
           5       0.95      0.97      0.96       892
           6       0.99      0.96      0.98       958
           7       0.98      0.96      0.97      1028
           8       0.95      0.96      0.96       974
           9       0.96      0.97      0.97      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


Testing: RMSprop


KeyboardInterrupt: 

In [ ]:
optimizers = ['SGD', 'Adam', 'RMSprop', 'Adagrad']
nn_models = {}
training_results = {}

for optimizer in optimizers:
    model = build_model()
    nn_models[optimizer] = model

    training_results[optimizer] = test_model(model, optimizer)